## **Parent Document Retriever**

문서를 검색할 때 **잘게 나누면 의미는 정확히 파악되지만, 맥락이 부족해지고**,  
**크게 나누면 맥락은 좋지만 의미가 흐려질 수 있음.**  
이 두 가지를 **균형 있게 해결**해주는 게  `ParentDocumentRetriever`

- Chunk 하기전 원본 문서

- Chunk된 문서와 함께 원본 문서가 필요 할 때 사용 할 수 있음

---

- 문서를 **작게 나눠서(청크)** 검색에 쓰고,  
- 검색된 청크가 속한 **큰 문서(부모 문서)**를 찾아  
- 더 풍부한 맥락과 내용을 제공

---

- **작은 단위로 정확하게 검색**  
- **큰 단위로 맥락까지 보장**  
- 문서 구조를 잘 활용해서 **검색 효율이 높아짐**



In [13]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [14]:
# 문서로드
loader = TextLoader('data/appendix-keywords.txt')
docs = loader.load()

In [ ]:
# 임베딩 
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

# 전체 문서 검색
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)

# vector store
vectorstore = Chroma(collection_name='full_documents', embedding_function=embeddings)   # 저장소 이름 : full_documents

# InMemoryStore
store = InMemoryStore()

# Retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,          # Chunking한 문서를 저장하는 저장소
    docstore=store,                   # 원본 문서를 저장하는 저장소
    child_splitter=child_splitter,
)

# Retriever에 문서 추가
retriever.add_documents(docs, ids = None)    # 원래 벡터스토어 에서 문서를 저장함 

### **Chunk에 대한 검색**

In [24]:
# 유사도 검색(vectorstore에 유사도 검색을 하면 청크를 검색하게됨)
sub_docs = vectorstore.similarity_search("Word2Vec")

# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


### **Parent(원본 문서)에 대한 검색**

In [29]:
# 문서를 검색
retrieved_docs = retriever.invoke("Word2Vec")

# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


### **더 큰 Chunk 의 크기를 조절**

전체 문서가 너무 크면 있는 그대로 검색하기에는 부적합

이런 경우, 실제로 우리가 하고 싶은 것은 먼저 원시 문서를 더 큰 청크로 분할한 다음, 더 작은 청크로 분할

그런 다음 작은 청크들을 인덱싱하지만, 검색 시에는 더 큰 청크를 검색

- `RecursiveCharacterTextSplitter`를 사용하여 부모 문서와 자식 문서를 생성
  - 부모 문서는 `chunk_size`가 1000으로 설정되어 있음
  - 자식 문서는 `chunk_size`가 200으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성


In [31]:
# Parent Docs Textsplitter
parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000)

# Child Docs Textsplitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)

# Child Chunk Indexing
vectorstore = Chroma(collection_name='split_parents', embedding_function=embeddings)

# InMemoryStore(부모문서 저장)
store = InMemoryStore()

# Retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter,
)

# 문서 추가
retriever.add_documents(docs) 

In [ ]:
# 유사도 검색
sub_docs = vectorstore.similarity_search("Word2Vec")
print(sub_docs[0].page_content)

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성


In [ ]:
# 문서 검색
retrieved_docs = retriever.invoke("Word2Vec")
print(retrieved_docs[0].page_content)

정의: 트랜스포머는 자연어 처리에서 사용되는 딥러닝 모델의 한 유형으로, 주로 번역, 요약, 텍스트 생성 등에 사용됩니다. 이는 Attention 메커니즘을 기반으로 합니다.
예시: 구글 번역기는 트랜스포머 모델을 사용하여 다양한 언어 간의 번역을 수행합니다.
연관키워드: 딥러닝, 자연어 처리, Attention

HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
